# Gypsum/anhydrite solubility in water

In this tutorial, we investigate the dependence of the sulfate mineral anhydrite (CaSO<sub>4</sub>) solubility in water for different temperatures and pressures.

> **Note**: In the databases available in Reaktoro, CaSO<sub>4</sub> is always referred to as anhydrite. When exposed to water, anhydrite transforms by the absorption of water to the more commonly known/occurring form gypsum (CaSO<sub>4</sub> · 2H<sub>2</sub>O).

First, we import necessary packages for the presented simulations:

In [ ]:
from reaktoro import *
import numpy as np
import math as math

First, we initialize the chemical system with aqueous, gaseous, and anhydrite phases:

In [ ]:
# Define the thermodynamic database
db =  SupcrtDatabase("supcrt98")

# Define the aqueous phase
aqueousphase = AqueousPhase(speciate("H O Ca S"), exclude("organic"))
aqueousphase.setActivityModel(ActivityModelHKF())

# Define the gaseous phase
minerals = MineralPhases("Anhydrite")

# Define the chemical system
system = ChemicalSystem(db, aqueousphase, minerals)

# Define the chemical properties
props = ChemicalProps(system)
aprops = AqueousProps(system)

# Defined the equilibrium solver
solver = EquilibriumSolver(system)

The anhydrite chemical reaction is defined based on the reaction equation and chemical system:

In [ ]:
reaction = db.reaction("Anhydrite = SO4-2 + Ca+2")

Below, function `water_problem()` defines the chemical problem corresponding to pure water (closed system):

In [ ]:
def water_problem(T, P):

    state = ChemicalState(system)
    state.temperature(T, "celsius")
    state.pressure(P, "bar")
    state.add("H2O(aq)", 1.0, "kg")

    return state

Below, we define the range of temperatures from 0 till 90 °C and calculate solubilities of anhydrite in water for the pressures P = 1, 100, 1000 bar. In a loop, we simulate the chemical equilibrium of 5 mol anhydrite in water. In addition, we also evaluate ph, ionic strength I, equilibrium constant log10(K),
and saturation index SI. If the SI > 0, the solution is supersaturated with anhydrite, whereas if the SI < 0, the solution is undersaturated with it. Finally, SI = 0 indicates equilibrium.

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["T", "P", "I", "pH", "SI", "log10K", "dAnhydrite"])

temperatures = np.arange(0.0, 91.0, 5.0) # in celsius
pressures = np.array([1.0, 100.0, 1000.0]) # in bar

for P in pressures:
    for T in temperatures:
        
        n0Anhydrite = 5.0

        # Equilibrate pure water
        state = water_problem(T, P)
        solver.solve(state)

        # Add 10mol of anhydrite
        state.set("Anhydrite", n0Anhydrite, "mol")

        # Equilibrate pure water with anhydrite
        solver.solve(state)

        # Update chemical and aqueous properties
        props.update(state)
        aprops.update(state)

        # Calculate ionic strength, pH, SI(Anhydrite) of the current state and log10K of the anhydrite dissolution reaction
        I = float(aprops.ionicStrength())
        pH = float(aprops.pH())
        SI = float(aprops.saturationIndex("Anhydrite"))

        log10K = float(reaction.props(T, "C", P, "bar").lgK)

        # Calculate the solubility of anhydrite
        dAnhydrite = n0Anhydrite - float(state.speciesAmount("Anhydrite"))
        #print(f"P = {P:.1f} bar, T = {T} C: "
        #      f"ph = {pH:.2f}, I = {I * 1e3:.2f} mmolal, lnK = {log10K:.4f}, SI = {SI:e}, dAnhydrite = {dAnhydrite:.4e}")

        # ["T", "P", "I", "pH", "SI", "log10K", "dAnhydrite"]
        df.loc[len(df)] = [T, P, I, pH, SI, log10K, dAnhydrite]

In [ ]:
df

The obtained solubility in water is 2.308049 g/L (25 &deg;C), which is considerably higher than for calcite. We use 172.17 g/mol as a molar mass of anhydrite.

In [ ]:
df_25C = df[df['T'] == 25.0] # fetch data corresponding to T = 25 celsius
df_25C_P1 = df_25C[df_25C['P'] == 1.0] # fetch data corresponding to T = 25 celsius and P = 1 bar
deltaAnhydrite = df_25C_P1['dAnhydrite'].iloc[0]
print(f"Solubility of anhydrite in water equals to {deltaAnhydrite:.6f} "
      f"mol/kgw = ... = {deltaAnhydrite * 0.17217 * 1e3:.6f} g/L")

According to the calculated values, the log10K at the 25 &deg;C is lnK = -9.9159, which corresponds to the database
values of `log_k = -4.36` (see, for example, `phreeqc.dat`). Below, we verify this from the numerical data collected to the dataframe `df`:

In [ ]:
df_25C = df[df['T'] == 25.0] # fetch data corresponding to T = 25 celsius
df_25C_P1 = df_25C[df_25C['P'] == 1.0] # fetch data corresponding to T = 25 celsius and P = 1 bar
deltaAnhydrite = df_25C_P1['dAnhydrite'].iloc[0]

lnK = -9.9159
log10K = lnK * math.log10(math.exp(1))
print("Anhydrite/gypsum logK = ", log10K)

Let us now plot solubilities of anhydrite as function of different temperatures for different pressure:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

output_notebook()
hovertool = HoverTool()
hovertool.tooltips = [("T", "@T"),
                      ("amount(Anhydrite) in brine", "@dAnhydrite mol")]

p = figure(
    title="SOLUBILITY OF ANHYDRITE",
    x_axis_label=r'TEMPERATURES [°C]',
    y_axis_label='AMOUNT OF DISSOLVED ANHYDRITE [MOL/KGW]',
    sizing_mode="scale_width",
    plot_height=300)

p.add_tools(hovertool)

colors = ['teal', 'darkred', 'indigo', 'coral']
for P, color in zip(pressures, colors):
    df_P = ColumnDataSource(df[df['P'] == P])
    p.line("T", "dAnhydrite", legend_label=f'{P} bar', line_width=3, line_cap="round", line_color=color, source=df_P)

p.legend.location = 'top_right'
show(p)

We see that anhydrite solubility decreases with increasing temperature. Increasing pressure also increases the solubility of calcium sulfate.